In [1]:
import pandas as pd
import numpy as np
from numpy import linalg
import os, sys

from utility.praser import deleteNAN
from matplotlib import pyplot as plt
from scipy import stats

In [3]:
# meta-data
data_types = ['cardboard', 'foam', 'metal', 'plastic', 'wooden-cabinet', 'wooden-shelf']
meas_numbers = 6
type_encoding = {
    'nothing':0,
    'cardboard':1, 
    'foam':2, 
    'metal':3, 
    'plastic':4, 
    'wooden-cabinet':5, 
    'wooden-shelf':6
}

In [5]:
# collecting all relevant files
df_list = []
pos_list = []
for data_type in data_types:
    for num in range(1,meas_numbers):
        # collect csv
        path = f"../../my_data/identification-dataset/nlos/anAn/{data_type}/data{num}/{data_type}-anAn-data{num}_data.csv"
        df = pd.read_csv(path)
        df['material'] = data_type
        # add material encoding
        df['encoded_material'] = type_encoding[df['material'].unique()[0]]
        # save df
        df_list.append(df)


        # collect pos
        folder = f"../../my_data/identification-dataset/nlos/anAn/{data_type}/data{num}/"
        # search for txt files in the folder
        for file in os.listdir(folder):
            if file.endswith(".txt"):
                pose_txt = os.path.join(folder, file)
       
        f = open(pose_txt,"r")
        pos=[];    quat=[]       # position and quaternion
        for line in f:
            x = line.split(",")
            if len(x) == 4:
                arr_x = [float(x[1]), float(x[2]), float(x[3])]
                pos.append(arr_x)
            if len(x) == 5:
                q_x = [float(x[1]), float(x[2]), float(x[3]), float(x[4])]
                quat.append(q_x)

        # calculate values
        pos = np.array(pos)     # [an1_p, an2_p, tag_p]
        quat = np.array(quat)   # [an1_quat, an2_quat, tag_quat]
        an1_p = pos[0,:]; 
        an2_p = pos[1,:]; 
        tag_p = pos[2,:]
        obs_up = pos[3:,:]
        obs_bt = np.copy(obs_up);  obs_bt[:,2] = 0 
        obstacle = np.concatenate((obs_up, obs_bt), axis=0)

        # add to list
        pos_list.append([pos, quat, an1_p, an2_p, tag_p, obs_up, obs_bt, obstacle])

In [7]:
# splitting and relabeling the data
splitted_df_list = []
for df, pos in zip(df_list, pos_list):

    # extract data
    # an1_tof = deleteNAN(np.array(df['an1_tof']))
    # an2_tof = deleteNAN(np.array(df['an2_tof']))

    an1_tof = np.array(df['an1_tof'])
    an2_tof = np.array(df['an2_tof'])

    an1_p = pos[2]
    an2_p = pos[3]

    # compute ground truth (gt)
    gt_an = linalg.norm(an1_p - an2_p) # distance between anchors

    # compute the error
    err_an1 = an1_tof - gt_an
    err_an2 = an2_tof - gt_an

    # add the error data
    df['err_an1'] = err_an1
    df['err_an2'] = err_an2
    splitted_df_list.append(df)

In [8]:
# builing the final df
df = pd.concat(splitted_df_list)
df.drop(columns=['material'])
df.dropna(inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59535 entries, 0 to 1975
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   tdoa12            59535 non-null  float64
 1   tdoa21            59535 non-null  float64
 2   snr_an1           59535 non-null  float64
 3   power_dif_an1     59535 non-null  float64
 4   snr_an2           59535 non-null  float64
 5   power_dif_an2     59535 non-null  float64
 6   an1_rx_snr        59535 non-null  float64
 7   an1_rx_powerdif   59535 non-null  float64
 8   an1_tof           59535 non-null  float64
 9   an2_rx_snr        59535 non-null  float64
 10  an2_rx_powerdif   59535 non-null  float64
 11  an2_tof           59535 non-null  float64
 12  material          59535 non-null  object 
 13  encoded_material  59535 non-null  int64  
 14  err_an1           59535 non-null  float64
 15  err_an2           59535 non-null  float64
dtypes: float64(14), int64(1), object(1)
memor

In [9]:
# storing the df
df.to_csv('anblock-error-dataset.csv', index=False,  encoding='utf-8')